## Part 4

Use the estimated transition and emission parameters, implement an algorithm to find the 7-th best
output sequences. 

In [1]:
import utils
import emission
import transition

In [2]:
dataset_folder = "data/AL/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [3]:
emission_data = emission.generate_emission_table(lines)
hashmap = emission_data["x_hashmap"]
word_freq = emission_data["x_word_freq"]
smoothed_hashmap = utils.add_unk(hashmap, word_freq, k=3)
emission_data["x_hashmap"] = smoothed_hashmap

x_vocab = utils.get_emission_vocab(smoothed_hashmap)

In [4]:
transition_pairs = transition.generate_transition_pairs(lines)

In [5]:
y_pairs = transition_pairs["Y_pairs"]
y_vocab = transition_pairs["y_vocab"]
y_freq = transition_pairs["y_freq"]

In [6]:
transition_data = transition.generate_transition_data(y_pairs, y_vocab)

In [7]:
import viterbi

In [8]:
hmm = viterbi.HMM()
hmm.fit_word_tokenizer(x_vocab)
hmm.fit_pos_tokenizer(y_vocab)
hmm.build_transition_weights(y_freq, transition_data)
hmm.build_emission_weights(emission_data)

In [9]:
train_data = dataset_folder + "dev.in"
lines = utils.read_file_to_lines(train_data)

sentences = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [token.strip() for token in sentence_xy]
    sentence = " ".join(words).strip()
    sentences.append(sentence)
    lines = lines[sentence_break+1:]

In [10]:
# only for the progress bar!
try:
    from tqdm import tqdm
    USE_TQDM = True
except Exception as e:
    print(e, "TQDM import error, disable progress bar")

if USE_TQDM:
    sentences_it = tqdm(sentences)
else:
    sentences_it = sentences

  0%|          | 0/1492 [00:00<?, ?it/s]

In [11]:
preds = []

for line in sentences_it:
    pred = hmm.viterbi_predict_k_best(line, k=7)
    pred = hmm.pos_tokens_to_labels(pred)
    preds.append(pred)
    
assert len(sentences) == len(preds)

100%|██████████| 1492/1492 [02:56<00:00,  8.44it/s]


In [12]:
outfile = dataset_folder + "dev.p4.out"

with open(outfile, "w") as f:
    for sentence, pred in zip(sentences, preds):
        word_array = sentence.split(" ")
        try:
            assert len(word_array) == len(pred)
            for i, word in enumerate(word_array):
                f.write(word + " " + pred[i] +"\n")
        except Exception as e:
            print(e)
            print(word_array)
            print(pred)
            break
        f.write("\n")

### Results

In [13]:
gold_data = dataset_folder + "dev.out"
pred_data = outfile

data = utils.run_eval(gold_data, pred_data)

print("Entity F:", data["entity_f"])
print("Entity precision:", data["entity_p"])
print("Entity recall:", data["entity_r"])
print("Sentiment F:", data["sentiment_f"])
print("Sentiment precision:", data["sentiment_p"])
print("Sentiment recall:", data["sentiment_r"])

Entity F: 0.6278
Entity precision: 0.6182
Entity recall: 0.6376
Sentiment F: 0.5521
Sentiment precision: 0.5437
Sentiment recall: 0.5608
